<font size="6"><b>SQL. Сервис для чтения книг по подписке.</b></font>

# Описание проекта

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 
Задача — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Краткий план работы и описание данных:

#### Описание данных:

**Таблица books**

Содержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

**Таблица publishers**

Содержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах на книги:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя пользователя, написавшего обзор;
* text — текст обзора.

#### Краткий план работы

Задания
* Посчитайте, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитайте количество обзоров и среднюю оценку;
* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
Как выполнить задание?
* Сделайте по одному SQL-запросу для решения каждого задания;
* Выведите результат каждого запроса в тетрадке;
* Опишите выводы по каждой из решённых задач.

# Подгрузка библиотек и самописные функции

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], db_config['pwd'], db_config['host'], db_config['port'],
db_config['db']) 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def qsl_request(query):
    display(pd.io.sql.read_sql(query, con = engine))

* Исследуйте таблицы — выведите первые строки;

In [3]:
qsl_request('SELECT * FROM books LIMIT 5')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
qsl_request('SELECT * FROM authors LIMIT 5')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
qsl_request('SELECT * FROM publishers LIMIT 5')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
qsl_request('SELECT * FROM ratings LIMIT 5')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
qsl_request('SELECT * FROM reviews LIMIT 5')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Cколько книг вышло после 1 января 2000 года


In [8]:
# возьмем вкючая первое число
qsl_request('''SELECT COUNT(DISTINCT book_id) 
            FROM books  
            WHERE CAST(publication_date AS DATE) >= '2000-01-01'
            ''')

,count
0,821


 После 1 января 2000 (включительно) была опубликованна 821 книга

### Расчет количества обзоров и средняя оценка для каждой книги

In [9]:
qsl_request('''SELECT avg_rating.book_id, books.title, count_reviews, avg_rating 
            FROM 
                (SELECT book_id, title 
                FROM books) AS books
            LEFT JOIN ( 
                SELECT book_id, AVG(rating) avg_rating 
                FROM ratings   
                GROUP BY book_id 
                ) AS avg_rating  ON avg_rating.book_id = books.book_id
            LEFT JOIN ( 
                SELECT book_id, COUNT(DISTINCT review_id) count_reviews 
                FROM reviews
                GROUP BY book_id 
               ) AS count_reviews  
            ON avg_rating.book_id = count_reviews.book_id 
            ''')

,book_id,title,count_reviews,avg_rating
0,652,The Body in the Library (Miss Marple #3),2.0,4.500000
1,273,Galápagos,2.0,4.500000
2,51,A Tree Grows in Brooklyn,5.0,4.250000
3,951,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,839,The Prophet,4.0,4.285714
...,...,...,...,...
995,64,Alice in Wonderland,4.0,4.230769
996,55,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,148,Christine,3.0,3.428571
998,790,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


Всего 1000 книг. Мы расчитали для них количество ревью и средний рейтинг

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
# возьмем 50 странциц включительно
# если мы делаем оговорку что издатель с наибольшим количеством книг может быть только один, 
qsl_request('''SELECT publisher, COUNT(*) count_published 
            FROM books 
            JOIN publishers ON books.publisher_id = publishers.publisher_id 
            WHERE num_pages >= 50 
            GROUP BY publisher 
            ORDER BY COUNT(*) DESC 
            LIMIT 1 
            ''')

,publisher,count_published
0,Penguin Books,42


In [11]:
# возьмем 50 странциц включительно
# так как издателей с наибольшим количеством книг может быть несколько, то используем макс, а не сортировку и лимит 1
qsl_request('''
            SELECT publisher, count_published 
            FROM 
                (SELECT publisher_id, COUNT(*) count_published 
                FROM books 
                WHERE num_pages >= 50 
                GROUP BY publisher_id) AS ONE 
            JOIN publishers ON ONE.publisher_id = publishers.publisher_id 
            WHERE count_published in  
                (SELECT MAX(count_published)  
                FROM \
                    (SELECT publisher_id, COUNT(*) count_published 
                    FROM books 
                    WHERE num_pages >= 50 
                    GROUP BY publisher_id) AS TWO)
            ''')

,publisher,count_published
0,Penguin Books,42


Мы выявили, что только один издатель издал больше всего книг - и это Penguin Books

### Автор с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)

In [12]:
# возьмем 50 оценок включительно
# делаем оговорку что автор с самой высокой средней оценкой книг может быть только один 
qsl_request('''SELECT author, AVG(book_rating) 
            FROM 
                (SELECT author, books.book_id, AVG(rating) book_rating 
                FROM books 
                JOIN authors ON books.author_id = authors.author_id 
                JOIN ratings ON books.book_id = ratings.book_id 
                WHERE books.book_id IN  
                    (SELECT book_id 
                    FROM 
                        (SELECT book_id, COUNT(rating) count_rating 
                        FROM ratings 
                        GROUP BY book_id 
                        HAVING COUNT(rating)>= 50) AS rc ) 
            GROUP BY author, books.book_id 
            ) AS aba 
            GROUP BY author 
            LIMIT 1 
            ''')

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Самые лучшие оценки у J.K. Rowling/Mary GrandPré

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
# возьмем 50 оценок включительно
qsl_request('''SELECT AVG(count_reviews) avg_count_reviews FROM 
                (SELECT username, COUNT(review_id) count_reviews 
                FROM reviews 
                WHERE username in (SELECT username FROM (SELECT username, COUNT(*) count_ratings  
                FROM ratings 
                GROUP BY username 
                HAVING COUNT(*) > 50) AS  cr) 
                GROUP BY username ) AS ur 
            ''')

,avg_count_reviews
0,24.333333


В среднем такие пользователи пишут 24 рецензии